## Import Libraries

In [1]:
import ee
import geopandas as gpd
import osmnx as ox
import os

print("Libraries imported successfully.")

try:
    ee.Initialize()
    print("GEE Initialized successfully.")
except Exception as e:
    print(f"Error initializing GEE: {e}")
    print("Make sure you have authenticated using ee.Authenticate() if this is your first time.")

Libraries imported successfully.
GEE Initialized successfully.


## Configuration

In [3]:
AOI_FILE = "../aoi.geojson"
EXPORT_DIR = "Coastal-Mapper-POC"
EXPORT_NAME = "S2_Composite_AOI"
BANDS = ['B2', 'B3', 'B4', 'B8'] # Blue, Green, Red, NIR
START_DATE = '2023-01-01'
END_DATE = '2023-12-31'
CLOUD_FILTER_PERCENT = 20 # Max cloud cover percentage

## 1. Load Area of Interest (AOI)

In [5]:
print(f"Loading AOI from {AOI_FILE}...")
aoi_gdf = gpd.read_file(AOI_FILE)

# Convert AOI to WGS84 and then to EE Geometry
aoi_wgs84 = aoi_gdf.to_crs(epsg=4326)
aoi_geometry_list = aoi_wgs84.geometry.iloc[0].__geo_interface__['coordinates']
aoi_ee = ee.Geometry.Polygon(aoi_geometry_list)
print("AOI loaded and converted to EE Geometry.")

Loading AOI from ../aoi.geojson...
AOI loaded and converted to EE Geometry.


## 2. Define export CRS

In [8]:
utm_crs = ox.project_gdf(gdf=aoi_wgs84).crs
EXPORT_CRS = utm_crs.to_string()
print(f"Export CRS set to {EXPORT_CRS}.")

Export CRS set to EPSG:32630.


## 3. Cloud Masking Function

In [9]:
def mask_s2_clouds_l2a(image):
    """
    Mask clouds in Sentinel-2 Level-2A images using the Scene Classification Layer (SCL).
    :param image: ee.Image
    :return: ee.Image with clouds masked out
    """
    scl = image.select('SCL')

    # valid SCL values for non-cloudy pixels
    good_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7)).Or(scl.eq(11))

    return image.updateMask(good_pixels)

## 4. Create Sentinel-2 Composite

In [11]:
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterDate(START_DATE, END_DATE)
                 .filterBounds(aoi_ee)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER_PERCENT))
                 .map(mask_s2_clouds_l2a)
                 .select(BANDS)
                )

# Generate median composite
composite_median = s2_collection.median()

# Clip composite to AOI
composite_clipped = composite_median.clip(aoi_ee)

print("Sentinel-2 composite created and clipped to AOI.")

Sentinel-2 composite created and clipped to AOI.


## 5. Export Composite to Google Drive

In [12]:
task_config = {
    'image': composite_clipped.toFloat(),
    'description': EXPORT_NAME,
    'folder': EXPORT_DIR,
    'scale': 10,  # 10m resolution (Important)
    'crs': EXPORT_CRS,
    'region': aoi_ee,
    'maxPixels': 1e10
}

In [13]:
task = ee.batch.Export.image.toDrive(**task_config)
task.start()

print(f"Export task started. Check your Google Drive folder '{EXPORT_DIR}' for the exported image.")

Export task started. Check your Google Drive folder 'Coastal-Mapper-POC' for the exported image.
